In [46]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

ERROR:root:Line magic function `%install_ext` not found.


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 27.1 ms


In [47]:
# read in known applicant files, dedupe and try to merge with applicants file
from collections import Counter
import difflib
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os

from data_cleaning_functions import (trans_remov_punc, standardize_whitespace, remove_punc, remove_suffix_from_last_name,
                                     clean_names, has_award, has_suffix, get_suffix, replace_last_name, 
                                     is_year_range, str_sim, clean_med_school, clean_std_college_name, long_form_date, 
                                    correct_mispellings)

from dev import (
    APP_DATA_DIR, SUM_STAT_DIR, ATT_DATA_DIR, CARD_DATA_DIR, CORRECTIONS_DIR, AWARDS_KEYWORDS, NAME_COLS, RAW_NAME_COLS, 
    RAW_CARD_ID, RAW_INDEX_IDS, PERSON_APPLICATION_ID, PERSON_ID, NIH_ID, FEMALE_FIRST_NAMES, FEMALE_MIDDLE_NAMES, 
    PICKLE_DIR)
from merging_functions import *

OUTPUT_CSV = False 

PERSONAL_INFO = [
    'clean_first_name', 'clean_last_name', 'clean_middle_name',
    'date_of_birth', 'medical_school', 'clean_college_trans']


time: 18.6 ms


In [48]:
# id column that links back to raw applicant data file
RAW_CARD_ID = 'raw_uuid'

# column where the raw id information is stored
RAW_INDEX_IDS = 'raw_card_ids'

# try to get one id per unique applicant in the dataset
PERSON_ID = 'person_uuid'
# id per deduped application-person - if someone applied multiple times, they will have multiple ids
PERSON_APPLICATION_ID = 'person_app_uuid' 
NIH_ID = 'dno'

APPLICANT_SUFFIX = '_ap'
ATTENDEE_SUFFIX = '_at'

%load_ext autoreload
%autoreload 2

%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 34.2 ms


In [49]:
apps_filename = 'index_cards_deduped_fuzzy.csv'
# apps_filename = 'person_application_date_wide.csv'

NIH_filename = 'unique_attendees.csv'
# NIH_filename = 'NIH_attendee_deduped_raw.csv'

time: 26.5 ms


In [50]:
NAME_COLS = ['clean_middle_name', 'clean_last_name', 'clean_first_name']

MED_TRAINING_COLS = ['res_dates', 'intern_dates', 'residency_hospital', 'internship_hospital', 'medical_school', 'residency']


time: 32 ms


In [51]:
# import NIH raw data set
NIH_raw = pd.read_csv(os.path.join(ATT_DATA_DIR, NIH_filename))

time: 57.4 ms


In [52]:
# drop from the data set all people with eod years > 1980
NIH = NIH_raw.loc[NIH_raw.eod_year<1980, :] 

time: 37.3 ms


In [53]:
# import cleaned, deduped applicant data set in wide form (multiple app dates as columns)
apps = pd.read_csv(os.path.join(APP_DATA_DIR, apps_filename))


time: 87.7 ms


In [54]:
apps.loc[:, 'res_dates'] = apps['residency_year(s)'].apply(long_form_date)

apps.loc[:, 'intern_dates'] = apps['internship_year(s)'].apply(long_form_date)

"Open"
Str. Medicine
Medicine
time: 64.9 ms


In [55]:
apps2 = apps.rename(
    columns={'residency_type': 'residency', 'internship_hospital_1': 'internship_hospital'})

time: 43.9 ms


In [56]:
NIH = NIH.rename(columns={'res_dtes':'res_dates', 'intern_dte':'intern_dates', 'intern_hos': 'internship_hospital', 
                         'res_hosp':'residency_hospital', 'clean_medical_school': 'medical_school'})
# sorted(NIH.columns)

time: 46 ms


In [57]:
string_med_cols = ['medical_school', 'residency_hospital', 'internship_hospital', 'residency']

# apply string cleaning function to each of the string medical info columns
apps2.loc[:, string_med_cols] = apps2[string_med_cols].applymap(clean_names)

time: 168 ms


In [58]:
to_remove = ['TERRECE', 'FRED', 'LAURENCE',
             'CUONO', 'DEFRENZE', 'JEFFERY', 'FINKLEMAN', 'SHERRAD', 'ANSCHNETZ', 'MARC', 'JENSON', 'KASTI', 
            'ADELBERT', 'RITCHARD', 'MANSFORD', 'DEFRENZO', 'DROBIN', 'HAMES', 'KREUZ', 'JERROLD', 'MANEUSI',
            'UNGARO']
to_replace = ['TERRENCE', 'FREDERICK', 'LAWRENCE',
              'CUOMO', 'DEFRONZO', 'JEFFREY', 'FINKELMAN', 'SHERRARD', 'ANSCHUETZ', 'MARCUS', 'JENSEN', 'KASTL',
              'ALBERT', 'RITCHARD', 'MANIFORD', 'DEFRONZO', 'DROBIS', 'JAMES', 'KRUEZ', 'JERROD', 'MANCUSI',
              'UNGARO']

correct_name_mispellings_fnc = funcy.rpartial(correct_mispellings, to_remove, to_replace)

apps2.loc[:, 'clean_last_name'] = apps2.clean_last_name.apply(correct_name_mispellings_fnc)
apps2.loc[:, 'clean_first_name'] = apps2.clean_first_name.apply(correct_name_mispellings_fnc)

time: 87.9 ms


In [59]:
# there are a bunch of duplicates in apps, where application year is the same, but first name is missing
name_dups = apps2.loc[
    apps2.duplicated(
        ['clean_last_name', 'medical_school', 'application_year'], keep=False), NAME_COLS+MED_TRAINING_COLS]

time: 33.2 ms


In [60]:
# name_dups.sort_values(['clean_last_name'])

time: 36.8 ms


In [61]:
apps3 = apps2.sort_values(['clean_last_name', 'clean_first_name']).drop_duplicates(
    ['clean_last_name', 'medical_school', 'application_year'])

time: 48.3 ms


In [62]:
# there are also some duplicates on middle and last name, but first is missing on one of the dups, 
# so we need to drop these
dups_mask = apps3.duplicated(['clean_last_name', 'clean_middle_name', 'application_year'], keep=False)

time: 31.9 ms


In [207]:
apps4 = apps3.loc[((~dups_mask) | ((dups_mask) & (~pd.isnull(apps3.clean_first_name)))), :]

time: 30.9 ms


In [208]:
print apps4.shape
print apps3.shape

(3982, 88)
(3988, 88)
time: 31.7 ms


In [209]:
apps4.loc[apps4.clean_last_name=='HAAKENSTAD', NAME_COLS]

,clean_middle_name,clean_last_name,clean_first_name
59,OTTO,HAAKENSTAD,ALAN


time: 47.1 ms


In [210]:
apps3.loc[apps3.clean_last_name=='HAAKENSTAD', NAME_COLS+['clean_medical_school', 'address', 'city']]

,clean_middle_name,clean_last_name,clean_first_name,clean_medical_school,address,city
59,OTTO,HAAKENSTAD,ALAN,NaN,1718 7th Street South,South Fargo


time: 37.9 ms


In [211]:
# function to go in and correct some of the name mispellings in both data sets
# MUTATING FUNCTION
def change_names(df, selection_type, selection_value, to_change_type, to_change_values):
    for t, v in zip(to_change_type, to_change_values):
        print t, v
        df.loc[df[selection_type]==selection_value, t] = v

time: 37.6 ms


In [212]:
%%capture
change_names(
    apps4, 'clean_last_name', 'CHESEBRO', ['clean_first_name', 'clean_middle_name'], ['BRUCE', 'WILCOX'])
change_names(
    apps4, 'clean_last_name', 'GALANTER', ['clean_first_name', 'clean_middle_name'], ['MARC', 'I'])
change_names(
    apps4, 'clean_last_name', 'BEAN', ['clean_first_name', 'clean_middle_name', 'medical_school'], ['SIDNEY', 'CHARLES', 'WAKE_FOREST'])
change_names(
    apps4, 'clean_last_name', 'EILER', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    apps4, 'clean_last_name', 'FALCHUK', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    apps4, 'clean_last_name', 'BOYD', ['clean_first_name', 'clean_middle_name'], ['MICHAEL', 'RAY'])
change_names(
    apps4, 'clean_last_name', 'CHAPMAN', 
    ['clean_first_name', 'clean_middle_name', 'medical_school'], ['STANLEY', 'WILLETS', 'ROCHESTER'])
change_names(apps4, 'clean_last_name', 'DANFORTH', ['clean_first_name'], ['DAVID'])
change_names(apps4, 'clean_last_name', 'HUNT', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'D'])
change_names(apps4, 'clean_last_name', 'KARK', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'ADRIAN'])
change_names(apps4, 'clean_last_name', 'KEBABIAN', ['clean_first_name', 'clean_middle_name'], ['JOHN', 'WILLIS'])
change_names(apps4, 'clean_last_name', 'KNOPF', ['clean_first_name', 'clean_middle_name'], ['HARRY', 'LOUIS'])
change_names(apps4, 'clean_last_name', 'KROLIKOWSKI', ['clean_first_name', 'clean_middle_name'], ['FRANCIS', 'JOHN'])
change_names(apps4, 'clean_last_name', 'KASTL', ['clean_first_name', 'clean_middle_name'], ['DAVID', 'GENE'])
change_names(apps4, 'clean_first_name', 'JAN', ['clean_last_name'], ['KNOWLER'])
change_names(apps4, 'clean_last_name', 'KLAVEMAN', ['clean_last_name'], ['KLAEVEMAN'])
change_names(apps4, 'clean_last_name', 'MATHEW', ['clean_last_name'], ['MATTHEW'])

apps4.loc[apps4.clean_last_name=='CHESEBRO', ['clean_first_name']] = 'BRUCE'
apps4.loc[apps4.clean_last_name=='CHESEBRO', ['clean_middle_name']] = 'WILCOX'
apps4.loc[(apps4.clean_last_name=='HEALY') & (apps4.medical_school=='USC KECK'), ['clean_first_name']] = 'MARK'
apps4.loc[(apps4.clean_last_name=='HEALY') & (apps4.medical_school=='USC KECK'), ['clean_middle_name']] = 'H'

time: 874 ms


In [213]:
print sorted(apps4.columns)
print sorted(NIH.columns)

['address', 'age', 'application_date', 'application_year', 'application_year_1', 'application_year_2', 'application_year_3', 'associate_program_entered', 'bob', 'ca', 'cc', 'citizenship', 'city', 'clean_college_trans', 'clean_first_initial', 'clean_first_name', 'clean_last_name', 'clean_middle_initial', 'clean_middle_name', 'clean_suffix', 'clinical', 'cord', 'date_of_birth', 'dbs', 'fifth', 'first_name', 'fuzzy_merge_col', 'honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third', 'ic', 'index', 'intern_dates', 'internship_hospital', 'internship_year(s)', 'is_female', 'is_match', 'last_name', 'last_name_counts', 'medical_school', 'medschool_year_grad', 'middle_name', 'nci', 'nei', 'nhi', 'nhli', 'niaid', 'niamd', 'niamdd', 'nichd', 'nichhd', 'nidr', 'niehs', 'nigms', 'nimh', 'nindb', 'ninds', 'oir', 'original_medical_school', 'other', 'person_uuid', 'pharm_ra', 'pi', 'ra', 'raw_uuid', 'raw_uuid_1', 'raw_uuid_2', 'raw_uuid_3', 'raw_uuid_4', 'r

In [214]:
apps4.loc[apps4.clean_last_name=='LENN', 'clean_first_name'] = 'NICHOLAS'
apps4.loc[apps4.clean_last_name=='BRESLOW', 'clean_first_name'] = 'JAN'
apps4.loc[apps4.clean_last_name=='BRESLOW', 'clean_middle_name'] = 'LESLIE'

time: 109 ms


In [215]:
apps4.loc[(apps4.clean_last_name=='NADLER') & (pd.isnull(apps4.clean_first_name)), 'clean_first_name'] = 'LEE'
apps4.loc[(apps4.clean_last_name=='NADLER') & (apps4.clean_first_name=='LEE'), 'clean_middle_name'] = 'MARSHALL'
apps4.loc[(pd.isnull(apps4.clean_first_name)) & (apps4.clean_last_name=='ROSEN'), 'clean_first_name'] = 'HENRY'

time: 119 ms


In [216]:
apps4.loc[(apps4.clean_last_name=='NEELON'), 'clean_first_name'] = 'FRANCIS'
apps4.loc[(apps4.clean_last_name=='NEELON') , 'clean_middle_name'] = 'ALBERT'

time: 87 ms


In [235]:
apps4.loc[(apps4.clean_last_name=='NICHOLAS') , 'clean_first_name']


1842        JOHN
2538    NICHOLAS
Name: clean_first_name, dtype: object

time: 53.9 ms


In [217]:
NIH.loc[(NIH.clean_last_name=='ALEXANDER') & (NIH.clean_first_name=='JOHN'), 'clean_middle_name'] = 'CHARLES'

time: 33.1 ms


In [218]:
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_middle_name'] = 'ARTHUR'
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_first_name'] = 'EDWARD'
NIH.loc[(
        NIH.clean_last_name=='LEBOWITZ') & (
            NIH.clean_first_name=='EDWARD'), 'medical_school'] = 'ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA'

time: 28.6 ms


In [219]:
NIH.loc[(NIH.clean_last_name=='NEELON'), NAME_COLS]

,clean_middle_name,clean_last_name,clean_first_name


time: 40.4 ms


In [220]:
# read in manual matches
man = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_dno_matches.xlsx'), index=False).rename(columns={'medical_school': 'dno_medical_school'})
man['medical_school'] = man.dno_medical_school.apply(clean_med_school)

time: 50.5 ms


In [221]:
apps4.loc[apps4.clean_last_name=='NEELON', NAME_COLS]

,clean_middle_name,clean_last_name,clean_first_name
3867,ALBERT,NEELON,FRANCIS


time: 38.4 ms


In [222]:
t = pd.merge(left=apps4, right=man, on=['clean_last_name', 'clean_middle_name'], how='inner', suffixes=['_x', '_y'])
t.shape

(107, 94)

time: 48.2 ms


In [223]:
t['sim'] = t[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

t['clean_first_name'] = t['clean_first_name_y']

t_1 = t.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')

time: 43.4 ms


In [227]:
# t[NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]
t_1.loc[t_1.duplicated('dno', keep=False), NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]

,clean_middle_name,clean_last_name,clean_first_name,dno,medical_school_x,medical_school_y,sim


time: 31.3 ms


In [228]:
um = man[~man.dno.isin(t_1.dno)]

um.shape

(6, 8)

time: 24.5 ms


In [229]:
t2 = pd.merge(left=apps4, right=um, on=['clean_last_name', 'clean_first_name'], how='inner', suffixes=['_x', '_y'])
t2.shape

(7, 94)

time: 38.6 ms


In [230]:
t2['sim'] = t2[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

time: 24.5 ms


In [231]:
t2['clean_middle_name'] = t2['clean_middle_name_y']

t2_1 = t2.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')
t2_1.shape

(6, 96)

time: 35.5 ms


In [245]:
t3 = pd.concat([t2_1[NAME_COLS+['dno', PERSON_ID]], t_1[NAME_COLS+['dno', PERSON_ID]]], axis=0)

time: 23.9 ms


In [246]:
man_dno = pd.merge(left=apps4, right=t3, on=PERSON_ID, how='left', suffixes=['_x', '_y'])
print man_dno.shape
print t3.shape
print man.shape

(3982, 92)
(103, 5)
(104, 8)
time: 40.5 ms


In [247]:
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno = man_dno.rename(columns={'clean_last_name_x': 'clean_last_name', 'clean_first_name_x': 'clean_first_name', 
                       'clean_middle_name_x': 'clean_middle_name'})
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno.loc[mask, 'clean_last_name'] = man_dno.loc[mask, 'clean_last_name_y']
mask = ~pd.isnull(man_dno.clean_first_name_y)
man_dno.loc[mask, 'clean_first_name'] = man_dno.loc[mask, 'clean_first_name_y']
mask = ~pd.isnull(man_dno.clean_middle_name_y)
man_dno.loc[mask, 'clean_middle_name'] = man_dno.loc[mask, 'clean_middle_name_y']

time: 53.2 ms


In [258]:
apps5 = man_dno.loc[pd.isnull(man_dno['dno']), :].drop('dno', axis=1) 

time: 33.9 ms


In [259]:
# remove females from data set
female_mask = (NIH.clean_first_name.isin(FEMALE_FIRST_NAMES)| NIH.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))  
NIH = NIH.loc[~female_mask, :]

female_mask = (apps5.clean_first_name.isin(FEMALE_FIRST_NAMES)| apps5.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))  
apps5 = apps5.loc[~female_mask, :]

time: 35.3 ms


In [260]:
def get_first_letter(str_var):
    if pd.isnull(str_var) or str_var=='':
        return np.nan
    return str_var[0]

time: 23.1 ms


In [261]:
NIH['clean_first_initial'] = NIH.clean_first_name.apply(get_first_letter)
NIH['clean_middle_initial'] = NIH.clean_middle_name.apply(get_first_letter)

time: 30.2 ms


In [262]:
print NIH.shape
print  apps5.shape

(3009, 33)
(3878, 91)
time: 26 ms


In [263]:
# After cleaning apps2 to match cleaning in Clean NIH Applicant notebook, we try to start merging
sims_cols = ['medical_school_sim', 'clean_middle_name_sim', 'clean_first_name_sim']

time: 21.2 ms


In [264]:
NIH['fuzzy_merge_col'] = NIH[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
apps5['fuzzy_merge_col'] = apps5[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
match1 = df_get_closest_matches(apps5, NIH, 'fuzzy_merge_col', suffixes=['_x', '_y']) 
print match1.shape

(3013, 122)
time: 4min 19s


In [265]:
# add last name counter to each
app_counter = Counter(apps4.clean_last_name.values)
NIH_counter = Counter(NIH.clean_last_name.values)
match1['last_name_counts_x'] = match1.clean_last_name_x.apply(lambda x: app_counter[x])
match1['last_name_counts_y'] = match1.clean_last_name_y.apply(lambda x: NIH_counter[x])


TypeError: ("'Series' objects are mutable, thus they cannot be hashed", u'occurred at index clean_last_name_y')

time: 90 ms


In [ ]:
def check_match(row):
    # address and application year match
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if row['clean_last_name_sim'] < 60 or app_eod_year_diff > 8:
        return 0
    
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 80:
        return 1
    if row['last_name_counts_x'] < 2 and row['last_name_counts_y'] < 2:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 5):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < .5:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 5) and row['medical_school_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 5) and row['clean_first_name_sim'] > 80:
        return 1
    return 0


feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'clean_last_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

match2 = add_similarity_features(match1, feature_dict, check_match, suffixes=['_x', '_y'])

In [ ]:
# match2.loc[(match2.is_match==2), sims_cols+['is_match', 'clean_first_name_x', 'clean_first_name_y',
#                         'clean_last_name_x', 'clean_last_name_y','medical_school_x', 'medical_school_y', PERSON_ID, NIH_ID]]

In [ ]:
# select out people who match and make sure each person id and dno only 1x in data set
match3 = match2[match2.is_match==1].sort_values(['clean_last_name_x']+sims_cols, ascending=False).dropna(
    axis=0, subset=[RAW_CARD_ID])
print match3.shape

In [ ]:
# match2[match2.clean_last_name_x=='ADLER']
NIH[NIH.clean_last_name=='ADLER']

In [ ]:
def filter_one_match_per_group(df, dedupe_col, sim_cols):
    # to merge cols should be a dict the names of the extra cols to merge in
    # values should be col names to rename
    # sim cols should be name of the columns to use as features
    # sim mask should be mask that accounts as actual mask
    # dedupe col is name of col to dedupe on

    def count_matches(id_list_arr):
        # for each id, make sure matched on 1x in data set
        # should be applied with rolling apply so takes in a dataframe and must return single value
        # unpack already matched ids from string
        current_id1 = id_list_arr[-1]
        other_matches = id_list_arr[:-1]
        is_dup = np.any(other_matches[:] == current_id1)
        if is_dup:
            return True
        return False

    # for each uuid, check for duplicates and choose best match based on sim cols
    # order of the sim cols should be with most important first
    dup_flag = '{}_duplicate'.format(dedupe_col)
    df[dup_flag] = 0
    df.loc[:, dup_flag] = df[
        dedupe_col].expanding(center=False, min_periods=0).apply(func=count_matches)

    df_matches = df[df['is_match'] == 1].sort_values([dedupe_col] + sim_cols, ascending=False)
    return df_matches.drop_duplicates([dedupe_col], keep='first')


In [ ]:
match4 = filter_one_match_per_group(match3, 'raw_uuid', sims_cols)
print match4.shape

In [ ]:
match5 = filter_one_match_per_group(match4, NIH_ID, sims_cols)
print match5.shape

In [ ]:
# get nonmatched NIH people and not matched applicants 
nm_apps = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match5[RAW_CARD_ID].dropna().values)
nm_NIH = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match5[NIH_ID].dropna().values)

In [ ]:
# do another round of matching just on last name
nm_match1 = df_get_closest_matches(nm_apps, nm_NIH, 'clean_last_name', suffixes=['_x', '_y']) 
print nm_match1.shape


In [ ]:
def check_match(row):
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if app_eod_year_diff > 8:
        return 0
    
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    max_name_sim = max(row['clean_first_name_sim'], row['medical_school_sim'], row['clean_middle_name_sim'])
    if max_name_sim < 60:
        return 0
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 60:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 6):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 40:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 6) and row['medical_school_sim'] > 80:
        return 1
    if not pd.isnull(row['medical_school_sim']) and row['medical_school_sim'] < 40:
        return 0
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_first_name_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_middle_name_sim'] > 80:
        return 1
    return 0

In [ ]:
nm_feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

nm_match2 = add_similarity_features(nm_match1, nm_feature_dict, check_match, suffixes=['_x', '_y'])


In [ ]:
nm_match3 = nm_match2.loc[(nm_match2.is_match==1) & (nm_match2.index!='MORTON'), :].reset_index(
    drop=False).rename(columns={'index': 'clean_last_name'})

In [ ]:
# append matches together
match6 = pd.concat([nm_match3, match5], axis=0)

In [ ]:
# get nonmatched NIH people and not matched applicants 
nm_apps2 = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match6[RAW_CARD_ID].dropna().values)
nm_NIH2 = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match6[NIH_ID].dropna().values)

In [ ]:
nm_NIH3 = nm_NIH2.loc[
    (nm_NIH2.eod_year< 1976) & (
        nm_NIH2.eod_year>1963), NAME_COLS+['medical_school', 'eod_year']].sort_values('clean_last_name')

In [ ]:
apps_match = nm_apps2.loc[nm_apps2.clean_last_name.isin(nm_NIH3.clean_last_name.values)]
test_merge = pd.merge(left=nm_NIH3, right=apps_match, on='clean_last_name', how='inner').sort_values('clean_last_name')
test_merge = test_merge[sorted(test_merge.columns)]
if OUTPUT_CSV:
    test_merge.to_csv(os.path.join(CORRECTIONS_DIR, 'test_merge_missing_NIH.csv'), index=False)

In [ ]:
match6.loc[pd.isnull(match6.clean_last_name), 'clean_last_name'] =  match6.loc[
    pd.isnull(match6.clean_last_name), 'clean_last_name_x']
to_drop = [c for c in match6.columns if c.endswith('_sim') or '_counts' in c]
match7= match6.drop(to_drop+[
        'dup_flag', 'eod_year_diff', 'fuzzy_merge_col_x', 'fuzzy_merge_col_y', 'unknown',
        'Unnamed: 0', 'raw_uuid_duplicate', 'dno_duplicate', 'count_missing',
                'clean_last_name_x', 'clean_last_name_y', 'is_match'], axis=1)

In [ ]:
# consolidate columns in match6
match7a = consolidate_merge_cols(match7, ['_x', '_y'], [])

In [115]:
match8 = pd.concat([
        man_dno[~pd.isnull(man_dno['dno'])], match7a, apps4.loc[~apps4[PERSON_ID].isin(match7a[PERSON_ID].values),:]], axis=0)
print sorted(match8.columns)
print match8.shape

['address', 'age', 'application_date', 'application_year', 'application_year_1', 'application_year_2', 'application_year_3', 'associate_program_entered', 'bob', 'ca', 'cc', 'citizenship', 'city', 'clean_college_trans', 'clean_first_initial', 'clean_first_name', 'clean_last_name', 'clean_middle_initial', 'clean_middle_name', 'clean_suffix', 'clinical', 'cord', 'data_source', 'date_of_birth', 'dbs', 'dno', 'dob', 'duplicate_dno', 'eod_year', 'fifth', 'first_name', 'fuzzy_merge_col', 'generation', 'honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third', 'ic', 'index', 'institute', 'intern_dates', 'internship_hospital', 'internship_year(s)', 'is_female', 'is_match', 'lab_brch', 'last_name', 'last_name_counts', 'level_0', 'med_school', 'medical_school', 'medschool_year_grad', 'middle_name', 'nci', 'nei', 'nhi', 'nhli', 'niaid', 'niamd', 'niamdd', 'nichd', 'nichhd', 'nidr', 'niehs', 'nigms', 'nimh', 'nindb', 'ninds', 'oir', 'original_medical_schoo

In [52]:
# replace date of birth with dob whenever date of birth missing and dob is not
match8.loc[
    (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'date_of_birth'] = match8.loc[
        (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'dob']


c1 = 'date_of_birth'
c2 = 'dob'
match8.loc[(pd.isnull(match8[c1])) & (~pd.isnull(match8[c2])), [c1, c2]]

,date_of_birth,dob


time: 31.1 ms


In [53]:
match9 = match8.drop(['dob'], axis=1)
print sorted(match9.columns)
print match9.shape

['address', 'age', 'application_date', 'application_year', 'application_year_1', 'application_year_2', 'application_year_3', 'associate_program_entered', 'bob', 'ca', 'cc', 'citizenship', 'city', 'clean_college_trans', 'clean_first_initial', 'clean_first_name', 'clean_last_name', 'clean_middle_initial', 'clean_middle_name', 'clean_suffix', 'clinical', 'cord', 'data_source', 'date_of_birth', 'dbs', 'dno', 'duplicate_dno', 'eod_year', 'fifth', 'first_name', 'fuzzy_merge_col', 'generation', 'honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third', 'ic', 'index', 'institute', 'intern_dates', 'internship_hospital', 'internship_year(s)', 'is_female', 'is_match', 'lab_brch', 'last_name', 'last_name_counts', 'level_0', 'med_school', 'medical_school', 'medschool_year_grad', 'middle_name', 'nci', 'nei', 'nhi', 'nhli', 'niaid', 'niamd', 'niamdd', 'nichd', 'nichhd', 'nidr', 'niehs', 'nigms', 'nimh', 'nindb', 'ninds', 'oir', 'original_medical_school', 'ot

In [54]:
match9['control_flag'] = 0
match9.loc[pd.isnull(match9.dno) & pd.isnull(match9.year_accepted), 'control_flag'] = 1

time: 15 ms


In [55]:
match9.rename(columns={'res_dates': 'residency_dates', 'intern_dates': 'internship_dates', 
                      'clean_college_trans': 'clean_college'}, inplace=True)

IMPORTANT_COLS = [NIH_ID, PERSON_ID, 'application_year', 'eod_year', 'application_date', 'clean_first_name', 'clean_middle_name', 
                 'clean_last_name', 'control_flag', 'year_accepted', 'rejected', 'rejection_date', 'clean_college', 'medical_school',
                'residency_dates', 'internship_dates']

other_cols = sorted([i for i in match9.columns if i not in IMPORTANT_COLS])

# order columns so important ones are 
match10 = match9[IMPORTANT_COLS+other_cols].sort_values(['clean_last_name', 'application_year'])

match11 = match10.dropna(subset=[PERSON_ID], axis=0).sort_values(['clean_last_name', 'clean_first_name'])

# wide_apps5.to_pickle(os.path.join(APP_DATA_DIR, 'all_apps_plus_NIH_info.p'))
match11.to_csv(os.path.join(APP_DATA_DIR, 'fuzzy_all_apps_plus_NIH_info.csv'), index=False)

wide_apps_v = match11.loc[(match11.application_year>1960) & (match11.application_year<1976), :].sort_values(
    ['clean_last_name', 'application_date'])

wide_apps_v.to_pickle(os.path.join(PICKLE_DIR, 'fuzzy_all_apps_plus_NIH_info_vietnam.p'))

wide_apps_v.to_csv(os.path.join(APP_DATA_DIR, 'fuzzy_all_apps_plus_NIH_info_vietnam.csv'), index=False)

time: 2.13 s


In [56]:
ln = 'BENDER'
wide_apps_v.loc[wide_apps_v.clean_last_name==ln, NAME_COLS+[PERSON_ID, 'medical_school']]

,clean_middle_name,clean_last_name,clean_first_name,person_uuid,medical_school
RICHARD ALAN BENDER,ALAN,BENDER,RICHARD,2784.0,UCLA


time: 32.3 ms
